## Import Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

## Load beautifulSoup

In [2]:
link = 'https://books.toscrape.com/'
res = requests.get(link)
soup = BeautifulSoup(res.text, 'html.parser')
soup.find('img').get('alt')

'A Light in the Attic'

## Extract data into first page

In [3]:
data= []
for sp in soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
    
    title = sp.find('img').get('alt')
    book_link  = 'https://books.toscrape.com/catalogue/' + sp.find('a').get('href')[10:]
    img_link = 'https://books.toscrape.com/' + sp.find('img').get('src')
    rating = sp.find('p').get('class')[-1]
    price = sp.find('p', class_ = 'price_color').text[1:]
    stock = sp.find('p', class_="instock availability").text.strip()
    
    data.append([title, rating, price, stock, book_link, img_link])

## Extract data All the first pages

In [4]:
data_1 = []
for i in tqdm(range(1,51)):
    
    link = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    for sp in soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
            
        title = sp.find('img').get('alt')
        book_link  = 'https://books.toscrape.com/catalogue/' + sp.find('a').get('href')
        img_link = 'https://books.toscrape.com/' + sp.find('img').get('src')
        rating = sp.find('p').get('class')[-1]
        price = sp.find('p', class_ = 'price_color').text[2:]
        stock = sp.find('p', class_="instock availability").text.strip()

        data_1.append([title, rating, price, stock, book_link, img_link])

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:36<00:00,  1.92s/it]


In [5]:
df = pd.DataFrame(data_1, columns = ['title', 'price', 'rating', 'stock', 'book_link', 'img_link'])
df.to_csv("books.csv", index = False)

In [6]:
data_2 = []

for i in tqdm(df['book_link']):
    
    res = requests.get(i)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    book_type = soup.find('ul', class_ = 'breadcrumb').find_all('a')[-1].text
    upc  = soup.find('table', class_ = 'table table-striped').find_all('td')[0].text
    price_excl_tax = soup.find('table', class_ = 'table table-striped').find_all('td')[2].text[2:] 
    price_incl_tax = soup.find('table', class_ = 'table table-striped').find_all('td')[3].text[2:]
    tax = soup.find('table', class_ = 'table table-striped').find_all('td')[4].text[2:]
    availability = soup.find('table', class_ = 'table table-striped').find_all('td')[5].text   
    reviews = soup.find('table', class_ = 'table table-striped').find_all('td')[6].text
    
    data_2.append([book_type, upc, price_excl_tax, price_incl_tax, tax, availability, reviews])

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [24:34<00:00,  1.47s/it]


In [14]:
df.

AttributeError: 'list' object has no attribute 'head'

In [13]:
df1 = pd.DataFrame(data_2, columns = ['book_type', 'upc', 'price_excl_tax', 'price_incl_tax', 'tax', 'availability', 'reviews'])
df1.head()

,book_type,upc,price_excl_tax,price_incl_tax,tax,availability,reviews
0,Poetry,a897fe39b1053632,51.77,51.77,0.00,In stock (22 available),0
1,Historical Fiction,90fa61229261140a,53.74,53.74,0.00,In stock (20 available),0
2,Fiction,6957f44c3847a760,50.10,50.10,0.00,In stock (20 available),0
3,Mystery,e00eb4fd7b871a48,47.82,47.82,0.00,In stock (20 available),0
4,History,4165285e1663650f,54.23,54.23,0.00,In stock (20 available),0
